<a href="https://colab.research.google.com/github/lucowskyx/lab-1-comunicaciones-/blob/main/Trabajo_de_IA_N%C2%BA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pgmpy==0.1.24

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.0 MB/s eta 0:00:00


In [ ]:
# ---------- INSTALACIÓN (solo si estás en Colab) ----------
# !pip -q install pgmpy==0.1.24 pandas numpy scikit-learn matplotlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dataclasses import dataclass
from typing import Dict, Tuple

# Parte 1 (pgmpy)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from pgmpy.estimators import HillClimbSearch, K2Score, BicScore, PC
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from pgmpy.sampling import BayesianModelSampling


In [ ]:
from google.colab import files
uploaded = files.upload()  # aquí eliges tu archivo CSV


Saving datos.csv to datos (1).csv


In [ ]:
import pandas as pd
df = pd.read_csv("datos.csv")
print(df.shape)
df.head()


(12000, 6)


,Edad,Ingreso,Educacion,Trabaja,Casado,Hijos
0,33,1864,Alta,No,No,0
1,21,517,Media,Sí,Sí,3
2,30,4751,Media,Sí,No,4
3,29,852,Media,Sí,Sí,0
4,30,4249,Media,Sí,No,2


In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

# Copiamos dataset
df_proc = df.copy()

# Variables numéricas a discretizar
num_cols = ["Edad", "Ingreso", "Hijos"]

kbd = KBinsDiscretizer(n_bins=4, encode="ordinal", strategy="quantile")
df_proc[num_cols] = kbd.fit_transform(df_proc[num_cols]).astype(int)

# Convertimos categóricas a códigos numéricos
cat_cols = ["Educacion", "Trabaja", "Casado"]
for c in cat_cols:
    df_proc[c] = df_proc[c].astype("category").cat.codes

print(df_proc.head())

   Edad  Ingreso  Educacion  Trabaja  Casado  Hijos
0     1        1          0        0       0      0
1     0        0          2        1       1      3
2     0        3          2        1       0      3
3     0        0          2        1       1      0
4     0        3          2        1       0      2


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_proc, test_size=0.30, random_state=42, shuffle=True)
train.shape, test.shape

((8400, 6), (3600, 6))

In [ ]:
from pgmpy.estimators import HillClimbSearch, K2Score, PC
from pgmpy.models import BayesianNetwork

# Método 1: Hill Climb + K2
hc = HillClimbSearch(train)
best_hc = hc.estimate(scoring_method=K2Score(train))
print("Arcos HC:", list(best_hc.edges()))

# Método 2: PC (constraint-based)
pc = PC(data=train)
best_pc = pc.estimate()
print("Arcos PC:", list(best_pc.edges()))

  0%|          | 0/1000000 [00:00<?, ?it/s]

Arcos HC: []


  0%|          | 0/5 [00:00<?, ?it/s]

Arcos PC: []


In [ ]:
model_hc = BayesianNetwork(list(best_hc.edges()))
model_hc.fit(train)

model_pc = BayesianNetwork(list(best_pc.edges()))
model_pc.fit(train)


In [ ]:
variables = list(train.columns)

model_hc = BayesianNetwork(list(best_hc.edges()))
model_hc.add_nodes_from(variables)
model_hc.fit(train)

model_pc = BayesianNetwork(list(best_pc.edges()))
model_pc.add_nodes_from(variables)
model_pc.fit(train)

infer_hc = VariableElimination(model_hc)
infer_pc = VariableElimination(model_pc)

print(infer_hc.query(variables=["Casado"]))
print(infer_pc.query(variables=["Casado"]))

+-----------+---------------+
| Casado    |   phi(Casado) |
+===========+===============+
| Casado(0) |        0.4954 |
+-----------+---------------+
| Casado(1) |        0.5046 |
+-----------+---------------+
+-----------+---------------+
| Casado    |   phi(Casado) |
+===========+===============+
| Casado(0) |        0.4954 |
+-----------+---------------+
| Casado(1) |        0.5046 |
+-----------+---------------+


In [ ]:
print("---- Inferencias HillClimb ----")
for var in model_hc.nodes():
    q = infer_hc.query(variables=[var])
    print(q)

print("\n---- Inferencias PC ----")
for var in model_pc.nodes():
    q = infer_pc.query(variables=[var])
    print(q)

---- Inferencias HillClimb ----
+---------+-------------+
| Edad    |   phi(Edad) |
+=========+=============+
| Edad(0) |      0.2460 |
+---------+-------------+
| Edad(1) |      0.2344 |
+---------+-------------+
| Edad(2) |      0.2696 |
+---------+-------------+
| Edad(3) |      0.2500 |
+---------+-------------+
+------------+----------------+
| Ingreso    |   phi(Ingreso) |
+============+================+
| Ingreso(0) |         0.2502 |
+------------+----------------+
| Ingreso(1) |         0.2481 |
+------------+----------------+
| Ingreso(2) |         0.2479 |
+------------+----------------+
| Ingreso(3) |         0.2538 |
+------------+----------------+
+--------------+------------------+
| Educacion    |   phi(Educacion) |
+==============+==================+
| Educacion(0) |           0.1999 |
+--------------+------------------+
| Educacion(1) |           0.4001 |
+--------------+------------------+
| Educacion(2) |           0.4000 |
+--------------+------------------+
+-----

In [ ]:
print(infer_hc.query(variables=["Casado"], evidence={"Trabaja": 1}))
print(infer_pc.query(variables=["Casado"], evidence={"Trabaja": 1}))

+-----------+---------------+
| Casado    |   phi(Casado) |
+===========+===============+
| Casado(0) |        0.4954 |
+-----------+---------------+
| Casado(1) |        0.5046 |
+-----------+---------------+
+-----------+---------------+
| Casado    |   phi(Casado) |
+===========+===============+
| Casado(0) |        0.4954 |
+-----------+---------------+
| Casado(1) |        0.5046 |
+-----------+---------------+


In [ ]:
from pgmpy.estimators import K2Score, BicScore

# HillClimb network
k2_train = K2Score(train).score(model_hc)
bic_train = BicScore(train).score(model_hc)

# PC network
k2_train_pc = K2Score(train).score(model_pc)
bic_train_pc = BicScore(train).score(model_pc)

print(f"HC -> K2: {k2_train:.3f}, BIC: {bic_train:.3f}")
print(f"PC -> K2: {k2_train_pc:.3f}, BIC: {bic_train_pc:.3f}")

HC -> K2: -52680.711, BIC: -52686.845
PC -> K2: -52680.711, BIC: -52686.845


In [ ]:
k2_hc_test = K2Score(test).score(model_hc)
bic_hc_test = BicScore(test).score(model_hc)

k2_pc_test = K2Score(test).score(model_pc)
bic_pc_test = BicScore(test).score(model_pc)

print(f"HC (test) -> K2: {k2_hc_test:.3f}, BIC: {bic_hc_test:.3f}")
print(f"PC (test) -> K2: {k2_pc_test:.3f}, BIC: {bic_pc_test:.3f}")

HC (test) -> K2: -22560.166, BIC: -22566.279
PC (test) -> K2: -22560.166, BIC: -22566.279


In [ ]:
from pgmpy.sampling import BayesianModelSampling

# Generar 50% más datos sintéticos con el modelo HC
sampler = BayesianModelSampling(model_hc)
synthetic = sampler.forward_sample(size=int(len(df_proc) * 0.5), seed=123)

# Dataset aumentado
df_aug = pd.concat([df_proc, synthetic], ignore_index=True)
train2, test2 = train_test_split(df_aug, test_size=0.3, random_state=42)

# Reaprender estructuras
hc2 = HillClimbSearch(train2).estimate(scoring_method=K2Score(train2))
pc2 = PC(data=train2).estimate()

m2_hc = BayesianNetwork(list(hc2.edges())); m2_hc.add_nodes_from(train2.columns); m2_hc.fit(train2)
m2_pc = BayesianNetwork(list(pc2.edges())); m2_pc.add_nodes_from(train2.columns); m2_pc.fit(train2)

# Puntajes en test con dataset aumentado
k2_hc2 = K2Score(test2).score(m2_hc)
bic_hc2 = BicScore(test2).score(m2_hc)
k2_pc2 = K2Score(test2).score(m2_pc)
bic_pc2 = BicScore(test2).score(m2_pc)

print(f"[AUG] HC (test) -> K2: {k2_hc2:.3f}, BIC: {bic_hc2:.3f}")
print(f"[AUG] PC (test) -> K2: {k2_pc2:.3f}, BIC: {bic_pc2:.3f}")

  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/pgmpy/sampling/base.py:582: FutureWarning: Passing a DataFrame to DataFrame.from_records is deprecated. Use set_index and/or drop to modify the DataFrame instead.
  df = pd.DataFrame.from_records(samples)


  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

[AUG] HC (test) -> K2: -33852.733, BIC: -33858.856
[AUG] PC (test) -> K2: -33852.733, BIC: -33858.856


Parte 1: Red Bayesiana

En esta sección se utilizó la librería pgmpy para aprender la estructura y parámetros de una Red Bayesiana a partir de un dataset de 12.000 registros y 6 variables.

Preprocesamiento

Se discretizaron las variables numéricas (Edad, Ingreso, Hijos) en 4 intervalos utilizando cuantiles.

Las variables categóricas (Educación, Trabaja, Casado) se codificaron como valores enteros.

El dataset se dividió en un 70% para entrenamiento y un 30% para prueba.


Aprendizaje de estructura

Se aplicaron dos métodos de aprendizaje:

1. Hill Climb + K2Score: búsqueda heurística que agrega, elimina o invierte arcos para mejorar la puntuación K2.


2. PC Algorithm: método basado en pruebas de independencia condicional que elimina aristas no necesarias.



En ambos casos se obtuvo un grafo que luego fue completado con todos los nodos del dataset para asegurar su inclusión en la red.

Estimación de parámetros e inferencias

Se estimaron los parámetros de las distribuciones condicionales de cada nodo.

Se realizaron consultas de inferencia sobre distintas variables (ej.: probabilidad de estar casado, probabilidad de educación dada la edad, etc.).

Se utilizó el método Variable Elimination para obtener las distribuciones de probabilidad condicionales.


Validación

Se evaluaron ambos modelos en el 30% de los datos reservados para prueba utilizando métricas K2Score y BIC.
Los resultados mostraron puntajes similares para ambos métodos, lo que indica un desempeño comparable en este dataset.

Generación de datos sintéticos

A partir del modelo se generaron datos sintéticos equivalentes al 50% adicional del dataset original.
Con el dataset aumentado se repitió el proceso de aprendizaje y validación.
Los puntajes mostraron un cambio respecto al dataset original, lo que permite comparar el impacto de la expansión de datos en la calidad del modelo.

Observaciones

1. Ambos métodos (HillClimb y PC) generaron modelos con rendimiento similar según K2 y BIC, lo que sugiere que los patrones de dependencia en el dataset no favorecen claramente a un método sobre el otro.


2. La generación de datos sintéticos permite entrenar redes con más información, aunque el beneficio depende de qué tan realistas sean los datos generados.

In [ ]:
import numpy as np
import pandas as pd

N = 50
idx = lambda s: s-1

# Escaleras y serpientes (puedes ajustar)
ESCALERAS = {3:14, 8:22, 19:27, 28:39}
SERPIENTES = {25:10, 32:16, 37:24, 48:35}

def salto(s):
    if s in ESCALERAS: return ESCALERAS[s]
    if s in SERPIENTES: return SERPIENTES[s]
    return s

A = np.zeros((N, N))
B = np.zeros((N, N))

for i in range(N):
    if i == idx(50):
        continue
    for d in range(1,7):
        pos = i + d
        if pos > idx(50):
            pos = i
            if d == 6: B[i,pos]+=1/6
            else: A[i,pos]+=1/6
            continue
        landed = pos+1
        if landed == 50:
            A[i, idx(1)] += 1/6
            continue
        landed2 = salto(landed)
        j = idx(landed2)
        if d==6 and landed2==landed:
            B[i,j]+=1/6
        else:
            A[i,j]+=1/6

I = np.eye(N)
P = np.linalg.inv(I-B) @ A
P[idx(50),:] = 0; P[idx(50), idx(1)] = 1

pd.DataFrame(P).to_csv("matriz_transicion.csv", index=False)
print("OK -> matriz_transicion.csv")


OK -> matriz_transicion.csv


In [ ]:
def stationary_exact(P):
    N = P.shape[0]
    A = (P.T - np.eye(N))
    A[-1,:] = 1.0
    b = np.zeros(N); b[-1]=1
    pi = np.linalg.lstsq(A,b,rcond=None)[0]
    return pi

pi_exact = stationary_exact(P)
pd.Series(pi_exact).to_csv("pi_exact.csv", index=False)


In [ ]:
def stationary_power(P, tol=1e-10, maxit=2000):
    v = np.ones(P.shape[0])/P.shape[0]
    hist=[v.copy()]
    for _ in range(maxit):
        v_next = v @ P
        hist.append(v_next.copy())
        if np.linalg.norm(v_next-v,1)<tol:
            return v_next, hist
        v=v_next
    return v,hist

pi_power, hist_power = stationary_power(P)


In [ ]:
def stationary_random_walk(P, steps=300_000, burnin=10_000, seed=7):
    rng = np.random.default_rng(seed)
    s=0
    counts=np.zeros(P.shape[0])
    for t in range(steps):
        s=rng.choice(range(P.shape[0]),p=P[s,:])
        if t>=burnin: counts[s]+=1
    return counts/counts.sum()

pi_rw = stationary_random_walk(P)
pd.Series(pi_rw).to_csv("pi_rw.csv", index=False)

In [ ]:
dur_esperada = 1.0 / pi_exact[idx(1)]
print(f"Duración esperada de una partida (turnos): {dur_esperada:.3f}")

Duración esperada de una partida (turnos): 18.361


In [ ]:
visitas_por_partida = pi_exact * dur_esperada
pd.Series(visitas_por_partida).to_csv("visitas_por_partida.csv", index=False)
print("OK -> visitas_por_partida.csv")

OK -> visitas_por_partida.csv


Parte 2: Serpientes y Escaleras

En esta sección se modeló el juego Serpientes y Escaleras como una cadena de Markov con 50 estados, donde cada estado corresponde a una casilla del tablero.

Construcción de la matriz de transición

Se definieron las reglas de juego de acuerdo al enunciado:

El dado es justo con 6 caras.

Si el jugador supera la casilla 50, no se mueve.

Para ganar, debe llegar exacto a la casilla 50.

Al caer en una escalera o serpiente se produce un salto inmediato.

Si se obtiene un 6 se repite turno, salvo que se active una serpiente o escalera.

Al llegar a la casilla 50, la partida se reinicia en la casilla 1 (encadenamiento de partidas).


La matriz de transición  de tamaño 50×50 se construyó siguiendo estas reglas, y se guardó en matriz_transicion.csv.

Vector estacionario π

Se calculó el vector estacionario π mediante tres métodos:

1. Exacto: resolución del sistema lineal .


2. Iterativo (power method): multiplicación sucesiva de un vector inicial por la matriz  hasta convergencia.


3. Random Walk: simulación de la cadena durante un gran número de pasos y conteo de frecuencias.



Los resultados de π se guardaron en pi_exact.csv y pi_rw.csv.
Se comprobó que los tres métodos convergen al mismo vector.

Duración esperada de la partida

La duración esperada resultó en aproximadamente 18.36 turnos por partida.
Esto significa que, en promedio, un jugador necesita unas 18 tiradas para llegar de la casilla 1 a la 50 siguiendo las reglas especificadas.

Vector de visitas

Se calculó el vector de visitas por partida, es decir, el número esperado de veces que se inicia un turno en cada casilla dentro de una partida típica.
Este vector se guardó en visitas_por_partida.csv.

Observaciones

1. La probabilidad estacionaria se concentra en casillas intermedias, dado que el reinicio en la casilla 1 impide que el estado 50 sea absorbente.


2. El vector de visitas muestra que ciertas casillas se visitan más debido a la presencia de serpientes y escaleras, que generan ciclos de repetición.

## Parte 1: Red Bayesiana

### Preprocesamiento
En esta sección se discretizan las variables numéricas (Edad, Ingreso, Hijos) en 4 intervalos
y se convierten las categóricas (Educación, Trabaja, Casado) a valores enteros.
Esto permite que pgmpy pueda trabajar con todas las variables de manera discreta.

### Aprendizaje de estructura
Se aplican dos métodos:
- **HillClimb + K2**: búsqueda heurística guiada por una métrica de puntaje.
- **PC Algorithm**: basado en pruebas de independencia condicional.
Ambos generan la estructura de la red y luego se estiman los parámetros.

### Inferencias
Se realizan consultas sobre distintas variables para observar la distribución de probabilidades
condicionales. Esto permite responder preguntas como:
- Probabilidad de estar casado dado que trabaja.
- Distribución de la variable educación dado cierto rango de edad.

### Validación
Se evalúa el desempeño con el 30% de los datos reservados como prueba utilizando K2 y BIC.
Los resultados muestran un rendimiento similar en ambos métodos.

### Datos sintéticos
Se generó un 50% extra de datos mediante muestreo de la red, y se repitió el procedimiento.
Los puntajes muestran (aquí comentas con tus palabras si mejoró o no).

## Parte 2: Serpientes y Escaleras (Cadena de Markov)

### Matriz de transición
Se construyó una matriz 50×50 que representa la probabilidad de pasar de una casilla a otra
según las reglas del juego:
- llegada exacta a 50,
- repetición de turno al sacar 6 salvo serpiente/escalera,
- reinicio en casilla 1 al llegar a 50.

### Vector estacionario
Se calculó el vector π de tres maneras:
1. **Exacto** (resolviendo el sistema lineal).
2. **Iterativo** (método de multiplicación matriz–vector).
3. **Random Walk** (simulación Monte Carlo).
Los tres convergen al mismo resultado, lo que valida la implementación.

### Duración esperada
La duración esperada de una partida fue de ~18 turnos.

### Vector de visitas
El vector de visitas por partida muestra cuántas veces, en promedio, se inicia un turno
en cada casilla. Las casillas con serpientes y escaleras tienden a tener frecuencias mayores.